In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [25]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [26]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [27]:
loss_fn = nn.CrossEntropyLoss()

In [28]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [29]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303346  [    0/60000]
loss: 2.297942  [ 6400/60000]
loss: 2.287188  [12800/60000]
loss: 2.282428  [19200/60000]
loss: 2.279191  [25600/60000]
loss: 2.258274  [32000/60000]
loss: 2.271457  [38400/60000]
loss: 2.255260  [44800/60000]
loss: 2.247299  [51200/60000]
loss: 2.228632  [57600/60000]
Test Error: 
 Accuracy: 37.9%, Avg loss: 0.035122 

Epoch 2
-------------------------------
loss: 2.243263  [    0/60000]
loss: 2.229669  [ 6400/60000]
loss: 2.202157  [12800/60000]
loss: 2.211887  [19200/60000]
loss: 2.204443  [25600/60000]
loss: 2.165064  [32000/60000]
loss: 2.204214  [38400/60000]
loss: 2.172452  [44800/60000]
loss: 2.164485  [51200/60000]
loss: 2.125782  [57600/60000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 0.033679 

Epoch 3
-------------------------------
loss: 2.159506  [    0/60000]
loss: 2.125522  [ 6400/60000]
loss: 2.070549  [12800/60000]
loss: 2.090484  [19200/60000]
loss: 2.064390  [25600/60000]
loss: 2.017733  [32000/600